In [1]:
"""
以下是推理结果，有几个warning，但推理结果和torch一致
"""
from mindnlp.transformers import CogVLMConfig,MLP,RMSNorm,PretrainedConfig,LlamaTokenizer,AutoModelForCausalLM
import mindspore
mindspore.set_seed(42)
ms_net = AutoModelForCausalLM.from_pretrained(
    'THUDM/cogvlm-chat-hf'
)
from PIL import Image
tokenizer = LlamaTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')
query = 'describe this picture'
image = Image.open('CogVLM.png').convert('RGB')
ms_inputs = ms_net.build_conversation_input_ids(tokenizer, query=query, history=[],images=[image])  # chat mode

ms_inputs = {
    'input_ids': ms_inputs['input_ids'].unsqueeze(0),
    'token_type_ids': ms_inputs['token_type_ids'].unsqueeze(0),
    'attention_mask': ms_inputs['attention_mask'].unsqueeze(0),
    'images': [[mindspore.Tensor(ms_inputs['images'][0],dtype=mindspore.float16)]],
}

gen_kwargs = {"max_length": 2048, "do_sample": False}
ms_out = ms_net.generate(**ms_inputs,**gen_kwargs)
outputs = ms_out[:, ms_inputs['input_ids'].shape[1]:]
print(tokenizer.decode(outputs[0]))

[WARNING] ME(12002:140052698522816,MainProcess):2024-05-14-19:50:24.413.323 [mindspore/run_check/_check_version.py:102] MindSpore version 2.2.13 and cuda version 11.8.89 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.
/hpc2hdd/home/ypeng455/.conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.702 seconds.
Prefix dict has been built successfully.
Loading checkpoint shards: 100%|██████████| 8/8 [01:06<00:00,  8.26s/it]
The following parameters in checkpoint files are not loaded:
['model.layers.0.self_attn.rotary_emb.inv_freq', 'm

The image is a flowchart that illustrates the process of how a visual language model and a visual expert built on a language model work. On the left, the visual language model takes an image and a textual description, processes them through an MLP adapter, and then through a VT encoder and a word embedding layer to produce a paired text. On the right, the visual expert model takes an image and a textual description, processes them through a multi-head attention mechanism with a QKV matrix, and then through a layer norm and text features to produce a text.</s>


In [6]:
query = '你好'
text_only_template = f"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {query} ASSISTANT:" 
query = text_only_template
image = Image.open('/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/CogVLM.png').convert('RGB')
ms_inputs = ms_net.build_conversation_input_ids(tokenizer, query=query, history=[]) # chat mode

ms_inputs = {
    'input_ids': ms_inputs['input_ids'].unsqueeze(0),
    'token_type_ids': ms_inputs['token_type_ids'].unsqueeze(0),
    'attention_mask': ms_inputs['attention_mask'].unsqueeze(0),
}

gen_kwargs = {"max_length": 2048, "do_sample": False}
ms_out = ms_net.generate(**ms_inputs,**gen_kwargs)
outputs = ms_out[:, ms_inputs['input_ids'].shape[1]:]
print(tokenizer.decode(outputs[0]))

你好! 很感谢您咨询. 请问您具备哪些��</s>


In [3]:
"""
这之后的内容是精度校验的记录,最差模块的精度在1e-4以内。
"""

import sys
print(sys.path)
from argparse import Namespace
from typing import TYPE_CHECKING, Optional, Tuple, List, Union, Literal, Dict, Any
import mindspore
import torch
import numpy as np
import modeling_cogvlm_pt
import visual_pt
from mindspore import ops
from einops import rearrange
from configuration_cogvlm_pt import CogVLMConfig as CogVLMConfigpt
from mindnlp.transformers import CogVLMConfig,MLP,RMSNorm,PretrainedConfig,LlamaTokenizer,AutoModelForCausalLM
from mindspore.amp import auto_mixed_precision
from PIL import Image
config_pt = CogVLMConfigpt()
config_pt = config_pt.from_dict(PretrainedConfig.get_config_dict(r'/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/THUDM/cogvlm-chat-hf')[0])
mindspore.set_seed(42)
torch.manual_seed(42)
config_ms = CogVLMConfig()
device = torch.device('cuda')
config_ms = config_ms.from_dict(PretrainedConfig.get_config_dict(r'/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/THUDM/cogvlm-chat-hf')[0])
vision_config = Namespace(**config_ms.vision_config)
LANGUAGE_TOKEN_TYPE = 0
VISION_TOKEN_TYPE = 1
mindspore.set_context(device_target='GPU')
mindspore.run_check()

# CogVLMForCausalLM
from mindnlp.transformers import CogVLMForCausalLM
ms_net = CogVLMForCausalLM(config_ms)
pt_net =  modeling_cogvlm_pt.CogVLMForCausalLM(config_pt).to('cuda')
pytorch_state_dict = pt_net.state_dict()

pt_dict = pt_net.state_dict()

pt_keys = list(pt_dict.keys())
for i in pt_keys:
    if 'inv_freq' in i:
        pt_keys.remove(i)
ms_keys = list(ms_net.parameters_dict().keys())
assert pt_keys == ms_keys
for key, parameter in ms_net.parameters_and_names():
    if 'inv' in key:
        continue
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).cpu().detach().numpy()))

from PIL import Image
tokenizer = LlamaTokenizer.from_pretrained('/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/lmsys/vicuna-7b-v1.5')
query = '你好！你可以用中文描述一下这张图片里有什么呢'
image = Image.open('/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/CogVLM.png').convert('RGB')
ms_inputs = ms_net.build_conversation_input_ids(tokenizer, query=query, history=[],images=[image])  # chat mode

ms_inputs = {
    'input_ids': ms_inputs['input_ids'].unsqueeze(0),
    'token_type_ids': ms_inputs['token_type_ids'].unsqueeze(0),
    'attention_mask': ms_inputs['attention_mask'].unsqueeze(0),
    'images': [[mindspore.Tensor(ms_inputs['images'][0])]],
}

gen_kwargs = {"max_length": 2048, "do_sample": False}
#ms_out = ms_net.generate(**ms_inputs,**gen_kwargs)
ms_out = ms_net(input_ids=ms_inputs['input_ids'],images=ms_inputs['images'],attention_mask=ms_inputs['attention_mask'],token_type_ids=ms_inputs['token_type_ids'])

from transformers import AutoModelForCausalLM, LlamaTokenizer,PretrainedConfig
tokenizer = LlamaTokenizer.from_pretrained('/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/lmsys/vicuna-7b-v1.5')

query = '你好！你可以用中文描述一下这张图片里有什么呢'
image = Image.open('/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/CogVLM.png').convert('RGB')
inputs = pt_net.build_conversation_input_ids(tokenizer, query=query, history=[],images=[image])  # chat mode
inputs = {
    'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
    'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
    'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
    'images': [[inputs['images'][0].to('cuda').to(torch.float32)]],
}

pt_out = pt_net(**inputs)


assert pt_out[0].shape == ms_out[0].shape
loss = 1e-4                                    # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out[0].cpu().detach().numpy(), ms_out[0].asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

['/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm', '/hpc2hdd/home/ypeng455/.conda/envs/py39/lib/python39.zip', '/hpc2hdd/home/ypeng455/.conda/envs/py39/lib/python3.9', '/hpc2hdd/home/ypeng455/.conda/envs/py39/lib/python3.9/lib-dynload', '', '/hpc2hdd/home/ypeng455/.conda/envs/py39/lib/python3.9/site-packages', '/hpc2hdd/home/ypeng455/mindnlp', '/tmp/tmpfb5wzsck']
MindSpore version:  2.2.13
The result of multiplication calculation is correct, MindSpore has been installed on platform [GPU] successfully!


In [4]:
# CLIP
from mindnlp.transformers import EVA2CLIPModel
ms_clip = EVA2CLIPModel(config_ms)
# ms_clip.to_float(mindspore.float16)
pt_clip =  modeling_cogvlm_pt.EVA2CLIPModel(config_pt).to('cuda')
pytorch_state_dict = pt_clip.state_dict()

pt_dict = pt_clip.state_dict()

pt_keys = list(pt_dict.keys())
for i in pt_keys:
    if 'inv_freq' in i:
        pt_keys.remove(i)
ms_keys = list(ms_clip.parameters_dict().keys())
assert pt_keys == ms_keys
for key, parameter in ms_clip.parameters_and_names():
    if 'inv' in key:
        continue
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).cpu().detach().numpy()))

In [5]:
input_id_np = np.random.rand(2,3,490,490)

input_id_ms = mindspore.Tensor(input_id_np,dtype=mindspore.float32)
input_id_pt = torch.Tensor(input_id_np).to(device).to(torch.float32)

ms_out = ms_clip(input_id_ms)
pt_out = pt_clip(input_id_pt)

assert pt_out[0].shape == ms_out[0].shape
loss = 1e-5                                    # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out.cpu().detach().numpy(), ms_out.asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

In [31]:
file_name_list = ['vision_transformer_attention_input','vision_transformer_attention_inner_out','vision_transformer_attention_inner_dense',
'vision_transformer_attention_output','vision_transformer_attention_layernorm','vision_transformer_hidden_states','vision_transformer_mlp_output',
'images','patch','transformer','slice','glu_linear','glu_norm','glu_act1','glu_act2','glu_act3','linear','cat']
ms_prefix = '_ms.npy'
pt_prefix = '_pt.npy'
pt_pp=''
pt_list = []
ms_list = []
for i,file in enumerate(file_name_list):
    
    ms_list.append(np.load(file+ms_prefix))
    pt_list.append(np.load(pt_pp+file+pt_prefix))


loss = 1e-4                                  # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
for i in range(len(ms_list)):
    print(file_name_list[i])
    assert np.allclose(ms_list[i],pt_list[i] , loss, loss) # 将结果全部转成array然后对比精度

vision_transformer_attention_input
vision_transformer_attention_inner_out
vision_transformer_attention_inner_dense
vision_transformer_attention_output
vision_transformer_attention_layernorm
vision_transformer_hidden_states
vision_transformer_mlp_output
images
patch
transformer
slice
glu_linear
glu_norm
glu_act1
glu_act2
glu_act3
linear
cat


In [15]:
# cogvlmDecoderLayer
from mindnlp.transformers import CogVLMDecoderLayer
ms_net = CogVLMDecoderLayer(config_ms)
pt_net = modeling_cogvlm_pt.CogVLMDecoderLayer(config_ms)
pt_net.to(device)

pytorch_state_dict = pt_net.state_dict()

pt_dict = pt_net.state_dict()

pt_keys = list(pt_dict.keys())
pt_keys.remove('self_attn.rotary_emb.inv_freq')
ms_keys = list(ms_net.parameters_dict().keys())
print(pt_keys)
print(ms_keys)
assert pt_keys == ms_keys
for key, parameter in ms_net.parameters_and_names():
    if 'inv' in key:
        continue
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).cpu().detach().numpy()))


file_name_list = ['hidden_states','token_type_ids','position_ids','attention_mask']
ms_prefix = '_ms.npy'
pt_prefix = '_pt.npy'
np_list = []
for i,file in enumerate(file_name_list):
    assert np.all(np.load(file+ms_prefix) == np.load(file+pt_prefix))
    np_list.append(np.load(file+ms_prefix))
ms_list = [mindspore.Tensor(i) for i in np_list]
pt_list = [torch.Tensor(i).to(device) for i in np_list]

ms_out = ms_net(ms_list[0],ms_list[1],ms_list[2],ms_list[3])
pt_out = pt_net(pt_list[0],pt_list[1].long(),pt_list[2].long(),pt_list[-1])

assert pt_out[0].shape == ms_out[0].shape
loss = 1e-5                                  # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out[0].cpu().detach().numpy(), ms_out[0].asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

['self_attn.vision_expert_query_key_value.weight', 'self_attn.vision_expert_dense.weight', 'self_attn.language_expert_query_key_value.weight', 'self_attn.language_expert_dense.weight', 'mlp.language_mlp.gate_proj.weight', 'mlp.language_mlp.up_proj.weight', 'mlp.language_mlp.down_proj.weight', 'mlp.vision_mlp.gate_proj.weight', 'mlp.vision_mlp.up_proj.weight', 'mlp.vision_mlp.down_proj.weight', 'input_layernorm.weight', 'post_attention_layernorm.weight']
['self_attn.vision_expert_query_key_value.weight', 'self_attn.vision_expert_dense.weight', 'self_attn.language_expert_query_key_value.weight', 'self_attn.language_expert_dense.weight', 'mlp.language_mlp.gate_proj.weight', 'mlp.language_mlp.up_proj.weight', 'mlp.language_mlp.down_proj.weight', 'mlp.vision_mlp.gate_proj.weight', 'mlp.vision_mlp.up_proj.weight', 'mlp.vision_mlp.down_proj.weight', 'input_layernorm.weight', 'post_attention_layernorm.weight']


In [2]:
# MLP test
ms_net = MLP(config_ms)
pt_net = modeling_cogvlm_pt.MLP(config_ms)

pytorch_state_dict = pt_net.state_dict()

pt_dict = pt_net.state_dict()
assert pt_dict.keys() == ms_net.parameters_dict().keys()
for key, parameter in ms_net.parameters_and_names():
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).detach().numpy()))

In [4]:
input_np = np.random.random((2,config_ms.hidden_size))
input_ms = mindspore.Tensor(input_np,dtype=mindspore.float32)
input_pt = torch.Tensor(input_np)
assert pt_net(input_pt).shape == ms_net(input_ms).shape
loss = 1e-5                                    # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_net(input_pt).detach().numpy(), ms_net(input_ms).asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

In [5]:
#RMSNorm test
ms_net = RMSNorm(config_ms.hidden_size,eps=config_ms.rms_norm_eps)
pt_net = modeling_cogvlm_pt.RMSNorm(config_ms.hidden_size,eps=config_ms.rms_norm_eps)

pytorch_state_dict = pt_net.state_dict()

pt_dict = pt_net.state_dict()
assert pt_dict.keys() == ms_net.parameters_dict().keys()
for key, parameter in ms_net.parameters_and_names():
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).detach().numpy()))

In [6]:
input_np = np.random.random((2,3,config_ms.hidden_size))
input_ms = mindspore.Tensor(input_np,dtype=mindspore.float32)
input_pt = torch.Tensor(input_np)
pt_out = pt_net(input_pt)
ms_out = ms_net(input_ms)
assert pt_out.shape == ms_out.shape
loss = 1e-5                                    # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out.detach().numpy(), ms_out.asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

In [7]:
#VisionExpertMLP Test
from mindnlp.transformers import VisionExpertMLP

ms_net = VisionExpertMLP(config_ms)
pt_net = modeling_cogvlm_pt.VisionExpertMLP(config_ms)

pytorch_state_dict = pt_net.state_dict()

pt_dict = pt_net.state_dict()
print(pt_dict.keys(),ms_net.parameters_dict().keys())

assert pt_dict.keys() == ms_net.parameters_dict().keys()
for key, parameter in ms_net.parameters_and_names():
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).detach().numpy()))

odict_keys(['language_mlp.gate_proj.weight', 'language_mlp.up_proj.weight', 'language_mlp.down_proj.weight', 'vision_mlp.gate_proj.weight', 'vision_mlp.up_proj.weight', 'vision_mlp.down_proj.weight']) odict_keys(['language_mlp.gate_proj.weight', 'language_mlp.up_proj.weight', 'language_mlp.down_proj.weight', 'vision_mlp.gate_proj.weight', 'vision_mlp.up_proj.weight', 'vision_mlp.down_proj.weight'])


In [8]:
batch = 2
length = 10
input_np = np.random.random((batch,length,config_ms.hidden_size))
input_ms = mindspore.Tensor(input_np,dtype=mindspore.float32)
input_pt = torch.Tensor(input_np)

input_id_np = np.random.randint(0,2,(batch,length))
input_id_ms = mindspore.Tensor(input_id_np,dtype=mindspore.int64)
input_id_pt = torch.Tensor(input_id_np)

pt_out = pt_net(input_pt,input_id_pt)
ms_out = ms_net(input_ms,input_id_ms)
assert pt_out.shape == ms_out.shape
loss = 1e-5                                    # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out.detach().numpy(), ms_out.asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

In [2]:
#RotaryEmbedding Test
from mindnlp.transformers import RotaryEmbedding

ms_net = RotaryEmbedding(config_ms.hidden_size / config_ms.num_attention_heads)
pt_net = modeling_cogvlm_pt.RotaryEmbedding(config_ms.hidden_size / config_ms.num_attention_heads)

pt_dict = pt_net.state_dict()
#assert pt_dict.keys() == ms_net.parameters_dict().keys()
for key, parameter in ms_net.parameters_and_names():
    if 'inv' in key:
        continue
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).detach().numpy()))

In [6]:
batch = 2
head = config_ms.num_attention_heads
head_size = int(config_ms.hidden_size / config_ms.num_attention_heads)
length = 10
input_np = np.random.random((batch,head,length,head_size))
input_ms = mindspore.Tensor(input_np,dtype=mindspore.float32)
input_pt = torch.Tensor(input_np)

pt_out = pt_net(input_pt,length)
ms_out = ms_net(input_ms,length)
assert pt_out[0].shape == ms_out[0].shape and pt_out[1].shape == ms_out[1].shape
loss = 1e-5                                    # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out[1].detach().numpy(), ms_out[1].asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

In [5]:
def rotate_half(x):
    x1, x2 = x[..., :x.shape[-1] // 2], x[..., x.shape[-1] // 2:]
    #return ops.cat((-x2, x1), dim=x1.ndim - 1)
    return ops.cat((-x2, x1), axis=x1.ndim - 1)
def apply_rotary_pos_emb_index_bhs(q, k, cos, sin, position_id,unsqueeze_dim=1):
    # batch_size, num_head, seq_len, hidden_size
    # cos, sin = F.embedding(position_id, cos.squeeze(1)).unsqueeze(1), \
    #     F.embedding(position_id, sin.squeeze(1)).unsqueeze(1)

    cos = cos.squeeze()
    sin = sin.squeeze()
    cos = cos[position_id].unsqueeze(unsqueeze_dim)
    sin = sin[position_id].unsqueeze(unsqueeze_dim)
    

    q, k = (q * cos) + (rotate_half(q) * sin), (k * cos) + (rotate_half(k) * sin)

    return q

input_position_np = [np.arange(length,dtype=int) for i in range(batch)]
input_position_ms = mindspore.Tensor(input_position_np,dtype=mindspore.int64)
apply_rotary_pos_emb_index_bhs(input_ms,input_ms,ms_out[0],ms_out[1],input_position_ms)[0][0][1]

Tensor(shape=[128], dtype=Float32, value= [-3.47945690e-02,  1.08714193e-01,  3.95617485e-01, -1.52103141e-01, -3.21287751e-01,  2.00536698e-01, -1.87772989e-01,  3.79675329e-02, -1.01648659e-01,  3.41640621e-01, -2.43581384e-02, -1.53884172e-01, 
  3.79164040e-01,  5.20352364e-01,  3.06936830e-01,  8.66513774e-02,  7.20749944e-02,  7.73576945e-02,  7.07507193e-01, -2.76955590e-03,  8.57028961e-01,  8.27848911e-01,  8.58387172e-01,  8.75291228e-01, 
  6.39051795e-01,  8.25882077e-01,  7.21081734e-01,  2.97383517e-01,  7.37333179e-01,  6.06829345e-01,  2.18471959e-02,  7.84013271e-01,  8.68759334e-01,  5.10558367e-01,  4.83114511e-01,  4.20871407e-01, 
  7.13463843e-01,  2.14443922e-01,  7.69713223e-01,  2.88738549e-01,  2.58968383e-01,  8.08105886e-01,  1.73843682e-01,  8.62305462e-01,  9.77771699e-01,  3.18609297e-01,  9.26611960e-01,  5.65522194e-01, 
  2.58579522e-01,  4.52274054e-01,  7.99573421e-01,  6.66201174e-01,  4.94637638e-01,  2.56956816e-01,  7.25732625e-01,  3.13210666e-0

In [7]:
#VisionExpertAttention Test
from mindnlp.transformers import VisionExpertAttention

ms_net = VisionExpertAttention(config_ms)
pt_net = modeling_cogvlm_pt.VisionExpertAttention(config_ms)

pytorch_state_dict = pt_net.state_dict()

pt_dict = pt_net.state_dict()
print(pt_dict.keys(),ms_net.parameters_dict().keys())

#assert pt_dict.keys() == ms_net.parameters_dict().keys()
for key, parameter in ms_net.parameters_and_names():
    if 'inv' in key:
        continue
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).detach().numpy()))

odict_keys(['rotary_emb.inv_freq', 'vision_expert_query_key_value.weight', 'vision_expert_dense.weight', 'language_expert_query_key_value.weight', 'language_expert_dense.weight']) odict_keys(['vision_expert_query_key_value.weight', 'vision_expert_dense.weight', 'language_expert_query_key_value.weight', 'language_expert_dense.weight'])


In [8]:
batch = 2
length = 64
input_np = np.random.random((batch,length,config_ms.hidden_size))
input_ms = mindspore.Tensor(input_np,dtype=mindspore.float32)
input_pt = torch.Tensor(input_np)

input_id_np = np.zeros((batch,length))
input_id_np[:,int(length/2):] = 1
input_id_ms = mindspore.Tensor(input_id_np,dtype=mindspore.int64)
input_id_pt = torch.LongTensor(input_id_np)

input_position_np = [np.arange(length,dtype=int) for i in range(batch)]
input_position_ms = mindspore.Tensor(input_position_np,dtype=mindspore.int64)
input_position_pt = torch.LongTensor(input_position_np)

input_attn_mask_np = np.random.randint(0,2,(batch,1,length,length))
input_attn_mask_ms = mindspore.Tensor(input_attn_mask_np,dtype=mindspore.int64)
input_attn_mask_pt = torch.Tensor(input_attn_mask_np)

pt_out = pt_net(input_pt,input_id_pt,input_position_pt,input_attn_mask_pt)
ms_out = ms_net(input_ms,input_id_ms,input_position_ms,input_attn_mask_ms)
assert pt_out[0].shape == ms_out[0].shape

loss = 1e-5                                 # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out[0].detach().numpy(), ms_out[0].asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

/tmp/ipykernel_5923/2922467035.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  input_position_pt = torch.LongTensor(input_position_np)


In [9]:
# cogvlmDecoderLayer
from mindnlp.transformers import CogVLMDecoderLayer
ms_net = CogVLMDecoderLayer(config_ms)
pt_net = modeling_cogvlm_pt.CogVLMDecoderLayer(config_ms)

pytorch_state_dict = pt_net.state_dict()

pt_dict = pt_net.state_dict()

pt_keys = list(pt_dict.keys())
pt_keys.remove('self_attn.rotary_emb.inv_freq')
ms_keys = list(ms_net.parameters_dict().keys())
print(pt_keys)
print(ms_keys)
assert pt_keys == ms_keys
for key, parameter in ms_net.parameters_and_names():
    if 'inv' in key:
        continue
    parameter.set_data(mindspore.Tensor(pt_dict.get(key).detach().numpy()))

['self_attn.vision_expert_query_key_value.weight', 'self_attn.vision_expert_dense.weight', 'self_attn.language_expert_query_key_value.weight', 'self_attn.language_expert_dense.weight', 'mlp.language_mlp.gate_proj.weight', 'mlp.language_mlp.up_proj.weight', 'mlp.language_mlp.down_proj.weight', 'mlp.vision_mlp.gate_proj.weight', 'mlp.vision_mlp.up_proj.weight', 'mlp.vision_mlp.down_proj.weight', 'input_layernorm.weight', 'post_attention_layernorm.weight']
['self_attn.vision_expert_query_key_value.weight', 'self_attn.vision_expert_dense.weight', 'self_attn.language_expert_query_key_value.weight', 'self_attn.language_expert_dense.weight', 'mlp.language_mlp.gate_proj.weight', 'mlp.language_mlp.up_proj.weight', 'mlp.language_mlp.down_proj.weight', 'mlp.vision_mlp.gate_proj.weight', 'mlp.vision_mlp.up_proj.weight', 'mlp.vision_mlp.down_proj.weight', 'input_layernorm.weight', 'post_attention_layernorm.weight']


In [10]:
batch = 2
length = 64
input_np = np.random.random((batch,length,config_ms.hidden_size))
input_ms = mindspore.Tensor(input_np,dtype=mindspore.float32)
input_pt = torch.Tensor(input_np)

input_id_np = np.zeros((batch,length))
input_id_np[:,int(length/2):] = 1
input_id_ms = mindspore.Tensor(input_id_np,dtype=mindspore.int64)
input_id_pt = torch.LongTensor(input_id_np)

input_position_np = [np.arange(length,dtype=int) for i in range(batch)]
input_position_ms = mindspore.Tensor(input_position_np,dtype=mindspore.int64)
input_position_pt = torch.LongTensor(input_position_np)

input_attn_mask_np = np.random.randint(0,2,(batch,1,length,length))
input_attn_mask_ms = mindspore.Tensor(input_attn_mask_np,dtype=mindspore.int64)
input_attn_mask_pt = torch.Tensor(input_attn_mask_np)

pt_out = pt_net(input_pt,input_id_pt,input_position_pt,input_attn_mask_pt)
ms_out = ms_net(input_ms,input_id_ms,input_position_ms,input_attn_mask_ms)
assert pt_out[0].shape == ms_out[0].shape

loss = 1e-5                                 # 精度误差一般为1e-5，最大为1e-3，必须小于1e-3否则迁移出错
assert np.allclose(pt_out[0].detach().numpy(), ms_out[0].asnumpy(), loss, loss) # 将结果全部转成array然后对比精度

In [10]:
def build_position_ids(x:"mindspore.Tensor(B, L)", attention_mask: Optional["mindspore.Tensor(B, L)"] = None) -> "mindspore.Tensor(B, L)":
    if attention_mask is not None:
        tmp = x.copy()
        tmp[~(attention_mask.bool())] = -1
    else:
        tmp = x.copy()
    # image boi eoi token as LANGUAGE_TOKEN_TYPE
    #is_boi_eoi = torch.zeros_like(x, dtype=torch.bool)
    is_boi_eoi = ops.zeros_like(x, dtype=mindspore.bool_)
    is_boi_eoi[:, 1:] |= (tmp[:, 1:] == VISION_TOKEN_TYPE) & (tmp[:, :-1] == LANGUAGE_TOKEN_TYPE)
    is_boi_eoi[:, 0] |= (tmp[:, 0] == VISION_TOKEN_TYPE)
    is_boi_eoi[:, :-1] |= (tmp[:, :-1] == VISION_TOKEN_TYPE) & (tmp[:, 1:] == LANGUAGE_TOKEN_TYPE)
    is_boi_eoi[:, -1] |= (tmp[:, -1] == VISION_TOKEN_TYPE)
    tmp[is_boi_eoi] = LANGUAGE_TOKEN_TYPE
    # final position ids
    # y = torch.zeros_like(x, dtype=mindspore.int64)
    y = ops.zeros_like(x, dtype=mindspore.int64)
    y[:, 1:] = (tmp[:, 1:] == LANGUAGE_TOKEN_TYPE) | ((tmp[:, 1:] == VISION_TOKEN_TYPE) & (tmp[:, :-1] == LANGUAGE_TOKEN_TYPE))
    y = y.cumsum(axis=-1)
    return y
out = build_position_ids(input_id_ms)
out[0]

Tensor(shape=[64], dtype=Int64, value= [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 
 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 
 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 34])